# Fine Tuning - Tech Twitter
See if we can get gpt to make fire tweets

## Part 1 - Scraping

First we need to get some tweets. We'll use this scraper I found for now.
Highly recommend you use a VPN, since this scraper is outside the Twitter API and they may rate limit/ban the IP.

In [2]:
import twint
import nest_asyncio
nest_asyncio.apply()

c = twint.Config()
c.Lang = 'en'
c.Limit = 1000
c.Username = 'joshterryplays'
c.Replies = False
c.Since = '2021-01-01'
c.Until = '2022-12-30'

c.Hide_output = True
c.Store_json = True
c.Output = 'data/joshterryplays2.jsonl'

twint.run.Search(c);

ModuleNotFoundError: No module named 'twint'

Load up the tweets & inspect them

In [30]:
import pandas as pd

pd.set_option('display.max_colwidth', 300)

lines = pd.read_json('data/tweets.jsonl', lines=True)
df = lines.sort_values(by='id', ascending=True)
cols = ['tweet', 'likes_count', 'retweets_count', 'replies_count', 'date']
df[df['reply_to'].map(len) == 0][cols].head()

,tweet,likes_count,retweets_count,replies_count,date
237,Not to mention that it’s a waste of time.,2,0,1,2022-12-23
236,Time spent wondering whether or not you’re doing everything you need to do is time spent not doing what needs to be done.,3,0,1,2022-12-23
235,You will always have to decide at some point that something is finished. You have to decide that you’re right or wrong and move on. You have to listen to a piece of advice and choose to take it or leave it.,2,0,1,2022-12-23
234,"And the more you own the fact that you have to make a decision and practice making decisions quickly. The more energetic, confident, and capable you will become. Because the indecisiveness itself and your fear that you’ll make the wrong choice is the real enemy.",2,0,1,2022-12-23
233,Not the occasional wrong decision.,3,0,1,2022-12-23


Grab the raw tweet data as it's own dataframe (it's nested under `raw_value`)

In [90]:
import openai
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

tweets = data.full_text.apply(lambda s: re.sub(r'https?://\S+', '', s)).to_frame()
tweets['tokens'] = tweets['full_text'].map(lambda s: len(tokenizer.encode(s)))
tweets.head()

,full_text,tokens
16,"Don’t think, “nobody would know.”\n\nBecause it turns you into a child that rebels against external rules.\n\nFor example, thinking, “Nobody would know if I gave up on this diet.”\n\nImplies that “other people” wanted you to be on the diet in the first place.\n\nBut who’s in charge? And…",87
2,"If you want to help someone, don’t tell them five things.\n🧵👇",22
21,Life requires work from you if you wish to survive.\n\nThe longer you fight it the more of a victim you will become.\n\nThe more you embrace it the easier the work will be.,41
15,Everyone is trying to use AI chatbots to remove the need for work.\n\nThe companies that win will be the ones that successfully integrate AI chatbots with their work.,35
11,AI is designed to remove work.\n\nBut the people who win will be the ones willing to do work…\n\nwith it.,28


In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")